# prop_logic

This module contains utilities for generates and validates propositional logic statement and proof.

In [ ]:
# | default_exp project.utils.prop_logic

In [ ]:
# | export

import random
import string


from dataclasses import (
    dataclass,
    field,
)
from enum import (
    Enum,
)
from typing import (
    List,
    Tuple,
)

In [ ]:
# | export


class Statement:
    def __str__(
        self,
    ):
        return "UNDEF"

    def __eq__(
        self,
        other,
    ):
        return False


@dataclass
class Atom(Statement):  # p0000, p0001, p0002, p0003, ..., p9999
    id: int

    def __str__(
        self,
    ):
        return "p{}".format(self.id)

    def __eq__(
        self,
        other,
    ):
        if type(other) != type(self):
            return False
        return self.id == other.id


@dataclass
class Not(Statement):  # !p0001
    atom: Statement

    def __str__(
        self,
    ):
        return f"!{self.atom}"

    def __eq__(
        self,
        other,
    ):
        if type(other) != type(self):
            return False
        return self.atom == other.atom


@dataclass
class And(Statement):  # p0001 \/ p0002
    atom_1: Statement
    atom_2: Statement

    def __str__(
        self,
    ):
        return f"({self.atom_1} & {self.atom_2})"

    def __eq__(
        self,
        other,
    ):
        if type(other) != type(self):
            return False
        return (
            self.atom_1,
            self.atom_2,
        ) == (
            other.atom_1,
            other.atom_2,
        )


@dataclass
class Or(Statement):  # p0001 /\ p0002
    atom_1: Statement
    atom_2: Statement

    def __str__(
        self,
    ):
        return f"({self.atom_1} | {self.atom_2})"

    def __eq__(
        self,
        other,
    ):
        if type(other) != type(self):
            return False
        return (
            self.atom_1,
            self.atom_2,
        ) == (
            other.atom_1,
            other.atom_2,
        )


@dataclass
class Impl(Statement):  # ap0001 -> p0002
    atom_1: Statement
    atom_2: Statement

    def __str__(
        self,
    ):
        return f"({self.atom_1} -> {self.atom_2})"

    def __eq__(
        self,
        other,
    ):
        if type(other) != type(self):
            return False
        return (
            self.atom_1,
            self.atom_2,
        ) == (
            other.atom_1,
            other.atom_2,
        )


@dataclass
class Iff(Statement):  # p0001 <-> p0002
    atom_1: Statement
    atom_2: Statement

    def __str__(
        self,
    ):
        return f"({self.atom_1} <-> {self.atom_2})"

    def __eq__(
        self,
        other,
    ):
        if type(other) != type(self):
            return False
        return (
            self.atom_1,
            self.atom_2,
        ) == (
            other.atom_1,
            other.atom_2,
        )


# class syntax
class J(
    str,
    Enum,
):
    Tautology = ""
    Axiom = "axiom"
    IIntroduction = "implication-introduction"
    CIntroduction = "conjution-introduction"
    DIntroduction1 = "disjunction-introduction"
    DIntroduction2 = "disjunction-introduction"
    NIntroduction = "negation-introduction"
    BIntroduction = "biconditional-introduction"
    IElimination = "implication-elimination"
    CElimination1 = "conjution-elimination"
    CElimination2 = "conjution-elimination"
    DElimination = "disjunction-elimination"
    NElimination = "negation-elimination"
    BElimination1 = "biconditional-elimination"
    BElimination2 = "biconditional-elimination"


@dataclass
class Inference:
    justify: J = field(default_factory=lambda: J.Tautology)
    indices: List[int] = field(default_factory=lambda: [])

    def __str__(
        self,
    ):
        idxs = [str(i) for i in self.indices] if self.indices else None
        return f"{self.justify.value} {','.join(idxs)}" if idxs else f"{self.justify.value}"


@dataclass
class Proof:
    statements: List[
        Tuple[
            int,
            Statement,
            Inference,
        ]
    ] = field(default_factory=lambda: [])
    numbers: int = 1000
    prepositions: int = 100

    def has(
        self,
        st,
    ):
        stsm = [s for _, s, _ in self.statements]
        return st in stsm

    def statement(
        self,
        i: int,
    ):
        (
            _,
            s,
            _,
        ) = self.statements[i]
        return s

    def size(
        self,
    ):
        return len(self.statements)

    def __str__(
        self,
    ):
        str_statements = []
        for (
            n,
            s,
            j,
        ) in self.statements:
            str_statements.append(f"{n}. {s}  [ {j} ] " if str(j) else f"{n}. {s}")
        return " nl\n".join(str_statements)

    def append_axiom(
        self,
        statement,
    ):
        n = len(self.statements)
        self.statements.append(
            (
                n,
                statement,
                Inference(J.Axiom),
            )
        )

    def raw_append(
        self,
        statement,
        justification=Inference(),
    ):
        n = len(self.statements)
        self.statements.append(
            (
                n,
                statement,
                justification,
            )
        )

    def _get_raw_axiom(
        self,
        r: Statement,
    ):
        return (
            r,
            Inference(J.Axiom),
        )

    def _get_raw_i_introduction(
        self,
        i: int,
        j: int,
    ):
        precond1 = self.statement(i)
        precond2 = self.statement(j)

        statement = Impl(
            precond1,
            precond2,
        )
        return (
            statement,
            Inference(
                J.IIntroduction,
                [
                    i,
                    j,
                ],
            ),
        )

    def _get_raw_i_elimination(
        self,
        i: int,
        j: int,
    ):  ## Modus ponen
        precond1 = self.statement(i)
        precond2 = self.statement(j)
        if type(precond1) != Impl:
            raise Exception("IElimination requieres an Implication as precondition")
        if precond2 != precond1.atom_1:
            raise Exception("IElimination P->Q  ... P")
        statement = precond1.atom_2
        return (
            statement,
            Inference(
                J.IElimination,
                [
                    i,
                    j,
                ],
            ),
        )

    def _get_raw_c_introduction(
        self,
        i: int,
        j: int,
    ):
        precond1 = self.statement(i)
        precond2 = self.statement(j)
        statement = And(
            precond1,
            precond2,
        )
        return (
            statement,
            Inference(
                J.CIntroduction,
                [
                    i,
                    j,
                ],
            ),
        )

    def _get_raw_c_elimination_1(
        self,
        i: int,
    ):  ## Modus ponen
        precond1 = self.statement(i)
        if type(precond1) != And:
            raise Exception("CElimination requieres an /\ as precondition")

        statement = precond1.atom_1
        return (
            statement,
            Inference(
                J.CElimination1,
                [i],
            ),
        )

    def _get_raw_c_elimination_2(
        self,
        i: int,
    ):  ## Modus ponen
        precond1 = self.statement(i)
        if type(precond1) != And:
            raise Exception("CElimination requieres an /\ as precondition")

        statement = precond1.atom_2
        return (
            statement,
            Inference(
                J.CElimination2,
                [i],
            ),
        )

    def _get_raw_d_introduction_1(
        self,
        r: Statement,
        i: int,
    ):
        precond = self.statement(i)
        statement = Or(
            r,
            precond,
        )
        return (
            statement,
            Inference(
                J.DIntroduction1,
                [i],
            ),
        )

    def _get_raw_d_introduction_2(
        self,
        r: Statement,
        i: int,
    ):
        precond = self.statement(i)
        statement = Or(
            precond,
            r,
        )
        return (
            statement,
            Inference(
                J.DIntroduction2,
                [i],
            ),
        )

    def _get_raw_d_elimination(
        self,
        p: Statement,
        i: int,
        j: int,
    ):  ## Modus ponen
        precond1 = self.statement(i)
        precond2 = self.statement(j)
        if type(precond1) != Impl:
            raise Exception("DElimination requires an Implication as first precondition")
        if type(precond2) != Impl:
            raise Exception("DElimination requires an Implication as second precondition")
        if precond1.atom_2 != precond2.atom_2:
            raise Exception("DElimination  P->Q  and R->Q as precondition")
        statement = Impl(
            Or(
                precond1.atom_1,
                precond2.atom_1,
            ),
            precond1.atom_2,
        )
        return (
            statement,
            Inference(
                J.DElimination,
                [
                    i,
                    j,
                ],
            ),
        )

    def _get_raw_n_introduction(
        self,
        i: int,
        j: int,
    ):
        precond1 = self.statement(i)
        precond2 = self.statement(j)
        if type(precond1) != Impl:
            raise Exception("NIntroduction requieres an Implication as first precondition")
        if type(precond2) != Impl:
            raise Exception("NIntroduction requieres an Implication as second precondition")

        if precond1.atom_1 != precond2.atom_1:
            raise Exception("NIntroduction requires P->Q  and P->!Q as precondition")

        if Not(precond1.atom_2) != precond2.atom_2:
            raise Exception("NIntroduction requires P->Q  and P->!Q as precondition")
        return (
            Not(precond1.atom_1),
            Inference(
                J.NIntroduction,
                [
                    i,
                    j,
                ],
            ),
        )

    def _get_raw_n_elimination(
        self,
        i: int,
    ):
        precond = self.statement(i)

        if type(precond) != Impl:
            raise Exception("NElimination requieres an Implication as first precondition")

        if precond.atom_1 != Not(precond.atom_2):
            raise Exception("NElimination  !P->P")
        return (
            precond.atom_2,
            Inference(
                J.NElimination,
                [i],
            ),
        )

    def _get_raw_b_introduction(
        self,
        i: int,
        j: int,
    ):
        precond1 = self.statement(i)
        precond2 = self.statement(j)
        if type(precond1) != Impl:
            raise Exception("BIntroduction requieres an Implication as first precondition")
        if type(precond2) != Impl:
            raise Exception("BIntroduction requieres an Implication as second precondition")

        if precond1.atom_1 != precond2.atom_2:
            raise Exception("BIntroduction requires P->Q  and Q->p as precondition")
        if precond1.atom_2 != precond2.atom_1:
            raise Exception("BIntroduction requires P->Q  and Q->p as precondition")
        return (
            Iff(
                precond1.atom_1,
                precond1.atom_2,
            ),
            Inference(
                J.BIntroduction,
                [
                    i,
                    j,
                ],
            ),
        )

    def _get_raw_b_elimination_1(
        self,
        i: int,
    ):
        precond = self.statement(i)
        if type(precond) != Iff:
            raise Exception("BElimination requieres an Iff as first precondition")
        return (
            Impl(
                precond.atom_1,
                precond.atom_2,
            ),
            Inference(
                J.NElimination,
                [i],
            ),
        )

    def _get_raw_b_elimination_2(
        self,
        i: int,
    ):
        precond = self.statement(i)
        if type(precond) != Iff:
            raise Exception("BElimination requieres an Iff as first precondition")
        return (
            Impl(
                precond.atom_2,
                precond.atom_1,
            ),
            Inference(
                J.NElimination,
                [i],
            ),
        )

    def get_random_statement(
        self,
    ):
        return Atom(
            random.randint(
                0,
                self.prepositions,
            )
        )  ## TODO: Get a better random statements generation

    def secure_apply(
        self,
        justification: Inference,
        simulate=False,
    ):
        if justification.justify == J.DIntroduction1 or justification.justify == J.DIntroduction2:
            return self.apply(
                justification,
                r=self.get_random_statement(),
                simulate=simulate,
            )
        else:
            return self.apply(
                justification,
                simulate=simulate,
            )

    def apply(
        self,
        justification: Inference,
        r: Statement = Statement(),
        simulate=False,
    ):
        i = justification.indices[0] if len(justification.indices) > 0 else 0
        j = justification.indices[1] if len(justification.indices) > 1 else 0
        (
            st,
            js,
        ) = (
            None,
            None,
        )
        if justification.justify == J.Axiom:
            (
                st,
                js,
            ) = self._get_raw_axiom(r)
        elif justification.justify == J.IIntroduction:
            (
                st,
                js,
            ) = self._get_raw_i_introduction(
                i,
                j,
            )
        elif justification.justify == J.CIntroduction:
            (
                st,
                js,
            ) = self._get_raw_c_introduction(
                i,
                j,
            )
        elif justification.justify == J.DIntroduction1:
            (
                st,
                js,
            ) = self._get_raw_d_introduction_1(
                r,
                i,
            )
        elif justification.justify == J.DIntroduction2:
            (
                st,
                js,
            ) = self._get_raw_d_introduction_2(
                r,
                i,
            )
        elif justification.justify == J.NIntroduction:
            (
                st,
                js,
            ) = self._get_raw_n_introduction(
                i,
                j,
            )
        elif justification.justify == J.BIntroduction:
            (
                st,
                js,
            ) = self._get_raw_b_introduction(
                i,
                j,
            )
        elif justification.justify == J.IElimination:
            (
                st,
                js,
            ) = self._get_raw_i_elimination(
                i,
                j,
            )
        elif justification.justify == J.CElimination1:
            (
                st,
                js,
            ) = self._get_raw_c_elimination_1(i)
        elif justification.justify == J.CElimination2:
            (
                st,
                js,
            ) = self._get_raw_c_elimination_2(i)
        elif justification.justify == J.DElimination:
            (
                st,
                js,
            ) = self._get_raw_d_elimination(
                r,
                i,
                j,
            )
        elif justification.justify == J.NElimination:
            (
                st,
                js,
            ) = self._get_raw_n_elimination(i)
        elif justification.justify == J.BElimination1:
            (
                st,
                js,
            ) = self._get_raw_b_elimination_1(i)
        elif justification.justify == J.BElimination2:
            (
                st,
                js,
            ) = self._get_raw_b_elimination_2(i)
        else:
            raise Exception(f"{justification.justify} not implemented")

        if not simulate:
            self.raw_append(
                st,
                js,
            )

        return (
            st,
            js,
        )

    def n_preconditions(
        self,
        just,
    ):
        return {
            J.Tautology: 0,
            J.Axiom: 0,
            J.IIntroduction: 1,
            J.CIntroduction: 1,
            J.DIntroduction1: 1,
            J.DIntroduction2: 1,
            J.NIntroduction: 2,
            J.BIntroduction: 2,
            J.IElimination: 2,
            J.CElimination1: 1,
            J.CElimination2: 1,
            J.DElimination: 2,
            J.NElimination: 1,
            J.BElimination1: 1,
            J.BElimination2: 1,
        }[just]

    def check(
        self,
        justify: J,
        i=None,
        j=None,
    ):
        if justify == J.Axiom:
            return True
        if justify == J.Tautology:
            return True

        if i != None:
            if justify == J.IIntroduction:
                return True
            elif justify == J.CIntroduction:
                return True
            elif justify == J.DIntroduction1:
                return True
            elif justify == J.DIntroduction2:
                return True
            elif justify == J.NIntroduction:
                precond1 = self.statement(i)
                if type(precond1) != Impl:
                    return False

                if j == None:
                    return True

                precond2 = self.statement(j)
                if type(precond2) != Impl:
                    return False
                if precond1.atom_1 != precond2.atom_1:
                    return False
                if Not(precond1.atom_2) != precond2.atom_2:
                    return False
                return True

            elif justify == J.BIntroduction:
                precond1 = self.statement(i)
                if type(precond1) != Impl:
                    return False

                if j == None:
                    return True

                precond2 = self.statement(j)
                if type(precond2) != Impl:
                    return False
                if precond1.atom_1 != precond2.atom_2:
                    return False
                if precond1.atom_2 != precond2.atom_1:
                    return False
                return True
            elif justify == J.IElimination:
                precond1 = self.statement(i)
                if type(precond1) != Impl:
                    return False

                if j == None:
                    return True

                precond2 = self.statement(j)
                if precond2 != precond1.atom_1:
                    return False
                return True

            elif justify == J.CElimination1:
                precond1 = self.statement(i)
                if type(precond1) != And:
                    return False
                return True
            elif justify == J.CElimination2:
                precond1 = self.statement(i)
                if type(precond1) != And:
                    return False
                return True
            elif justify == J.DElimination:
                precond1 = self.statement(i)

                if type(precond1) != Impl:
                    return False

                if j == None:
                    return True

                precond2 = self.statement(j)
                if type(precond2) != Impl:
                    return False
                if precond1.atom_2 != precond2.atom_2:
                    return False
                return True
            elif justify == J.NElimination:
                precond = self.statement(i)
                if type(precond) != Impl:
                    return False
                if precond.atom_1 != Not(precond.atom_2):
                    return False
                return True
            elif justify == J.BElimination1:
                precond = self.statement(i)
                if type(precond) != Iff:
                    return False
                return True
            elif justify == J.BElimination2:
                precond = self.statement(i)
                if type(precond) != Iff:
                    return False
                return True
            else:
                raise Exception(f"{justify} not implemented")
        else:
            False
        return False

    def generate_step(
        self,
    ):
        inference_options = [
            J.IIntroduction,
            J.CIntroduction,
            J.DIntroduction1,
            J.DIntroduction2,
            J.NIntroduction,
            J.BIntroduction,
            J.IElimination,
            J.CElimination1,
            J.CElimination2,
            J.DElimination,
            J.NElimination,
            J.BElimination1,
            J.BElimination2,
        ]
        tryouts = 100
        for _ in range(tryouts):
            just = random.choice(inference_options)
            n_precond = self.n_preconditions(just)
            if n_precond == 0:
                (
                    st,
                    js,
                ) = self.secure_apply(
                    Inference(
                        just,
                        [],
                    ),
                    simulate=True,
                )
                if not self.has(st):
                    self.raw_append(
                        st,
                        js,
                    )
                    return
                else:
                    continue  ## abort current

            n_statements = self.size()
            idxs = list(range(n_statements))
            random.shuffle(idxs)

            i = -1
            for k in idxs:
                if self.check(
                    just,
                    k,
                ):
                    i = k
                    break
            if i == -1:
                continue
            if n_precond == 1:
                (
                    st,
                    js,
                ) = self.secure_apply(
                    Inference(
                        just,
                        [i],
                    ),
                    simulate=True,
                )
                if not self.has(st):
                    self.raw_append(
                        st,
                        js,
                    )
                    return
                else:
                    continue

            j = -1
            idxs = list(range(n_statements))
            for k in idxs:
                if self.check(
                    just,
                    i,
                    k,
                ):
                    j = k
                    break
            if j == -1:
                continue

            (
                st,
                js,
            ) = self.secure_apply(
                Inference(
                    just,
                    [
                        i,
                        j,
                    ],
                ),
                simulate=True,
            )
            if not self.has(st):
                self.raw_append(
                    st,
                    js,
                )
                return
            else:
                continue
        raise Exception(f"Didnt find any inference to apply after {tryouts} intents")

    def generate_steps(
        self,
        n,
    ):
        for _ in range(n):
            self.generate_step()

In [ ]:
# | export
from lark import (
    Lark,
    Transformer,
    v_args,
)

# Grammar in Lark Format
grammar = """
    start: expr
    expr:  impl                 -> forward 
    
    ?impl:  binop                 -> forward
          | impl "->" binop      -> imp_ 
          | impl "<->" binop     -> iff_
          
         
    ?binop: batom              -> forward
           |  binop "&" batom     -> and_
           |  binop "|" batom     -> or_
          

    ?batom:  "p" NUMBER           -> var_
           | "!" batom          -> not_
           | "(" expr ")"      -> parens_    
    
    NUMBER: /-?\d+(\.\d+)?/
    
    %import common.WS
    %ignore WS
"""


class PropLogicTransformer(Transformer):
    def forward(
        self,
        value,
    ):
        return value[0]

    def var_(
        self,
        value,
    ):
        return Atom(int(value[0]))

    def not_(
        self,
        value,
    ):
        return Not(value[0])

    def and_(
        self,
        values,
    ):
        return And(
            values[0],
            values[1],
        )

    def or_(
        self,
        values,
    ):
        return Or(
            values[0],
            values[1],
        )

    def parens_(
        self,
        value,
    ):
        return value[0]

    def iff_(
        self,
        values,
    ):
        return Iff(
            values[0],
            values[1],
        )

    def imp_(
        self,
        values,
    ):
        return Impl(
            values[0],
            values[1],
        )


# Using the Lark parser with the defined grammar and transformer
lark_parser = Lark(
    grammar,
    parser="lalr",
    transformer=PropLogicTransformer(),
)


def parse_logic(
    statement,
):
    obj = lark_parser.parse(statement).children[0]
    return obj

In [ ]:
# | hide
print(f"{parse_logic('! p1')}")
print(f"{parse_logic('!p1->p2')}")
print(f"{parse_logic('!(p1->p2)')}")
print(f"{parse_logic('p1 & p2')}")
print(f"{parse_logic('p1 | p2')}")
print(f"{parse_logic('(p1 & p2) | (p3 & p4)')}")
print(f"{parse_logic('(p1 & p2) -> (p3 & p4)')}")
print(f"{parse_logic('(p1 & p2) <-> (p3 & p4)')}")

In [ ]:
proof = Proof()
proof.append_axiom(parse_logic("p0001"))
proof.append_axiom(parse_logic("p0002"))
proof.append_axiom(parse_logic("p0003->p0004"))
proof.append_axiom(parse_logic("p0003"))
proof.apply(
    Inference(
        J.IIntroduction,
        [
            0,
            1,
        ],
    )
)
proof.apply(
    Inference(
        J.IElimination,
        [
            2,
            3,
        ],
    )
)
proof.apply(
    Inference(
        J.CIntroduction,
        [
            0,
            1,
        ],
    )
)
proof.apply(
    Inference(
        J.CElimination1,
        [6],
    )
)
proof.apply(
    Inference(
        J.CElimination2,
        [6],
    )
)
proof.apply(
    Inference(
        J.DIntroduction1,
        [3],
    ),
    parse_logic("p0008"),
)
proof.apply(
    Inference(
        J.DIntroduction2,
        [3],
    ),
    parse_logic("p0008"),
)
proof.append_axiom(parse_logic("p0003->p0004"))
proof.append_axiom(parse_logic("p0005->p0004"))
proof.apply(
    Inference(
        J.DElimination,
        [
            11,
            12,
        ],
    )
)
proof.append_axiom(parse_logic("p0003->p0004"))
proof.append_axiom(parse_logic("p0003->!p0004"))
proof.apply(
    Inference(
        J.NIntroduction,
        [
            14,
            15,
        ],
    )
)
proof.append_axiom(parse_logic("!p0003->p0003"))
proof.apply(
    Inference(
        J.NElimination,
        [17],
    )
)
proof.append_axiom(parse_logic("p0003->p0004"))
proof.append_axiom(parse_logic("p0004->p0003"))
proof.apply(
    Inference(
        J.BIntroduction,
        [
            19,
            20,
        ],
    )
)
proof.apply(
    Inference(
        J.BIntroduction,
        [
            20,
            19,
        ],
    )
)
proof.apply(
    Inference(
        J.BElimination1,
        [21],
    )
)
proof.apply(
    Inference(
        J.BElimination2,
        [21],
    )
)
proof.apply(
    Inference(
        J.IIntroduction,
        [
            12,
            13,
        ],
    )
)
proof.apply(
    Inference(
        J.IIntroduction,
        [
            13,
            14,
        ],
    )
)
proof.apply(
    Inference(
        J.IIntroduction,
        [
            25,
            26,
        ],
    )
)


print(proof)

In [ ]:
# Generator

for _ in range(5):
    proof = Proof()
    proof.append_axiom(parse_logic("p1"))
    proof.append_axiom(parse_logic("p2"))
    proof.append_axiom(parse_logic("!p3"))
    proof.append_axiom(parse_logic("p1->(p2|p3)"))
    proof.generate_steps(20)
    print(proof)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()